In [2]:
pwd

'/Users/yamadaikuya/pythonista/toto'

In [1]:
###LOGISTIC REDRESSION　の練習台
import pandas as pd
import numpy as np
from pandas import DataFrame
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
%matplotlib inline
# ファイルを読み込む
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [2]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [3]:
test_df.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [3]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
#利用しない変数は削除します
train_df = train_df.drop(['Name','SibSp','Parch','Ticket','Fare','Cabin','Embarked'],axis=1)
test_df = test_df.drop(['Name','SibSp','Parch','Ticket','Fare','Cabin','Embarked'],axis=1)

In [7]:
train_df.head()

,PassengerId,Survived,Pclass,Sex,Age
0,1,0,3,male,22.0
1,2,1,1,female,38.0
2,3,1,3,female,26.0
3,4,1,1,female,35.0
4,5,0,3,male,35.0


In [8]:
test_df.head()

,PassengerId,Pclass,Sex,Age
0,892,3,male,34.5
1,893,3,female,47.0
2,894,2,male,62.0
3,895,3,male,27.0
4,896,3,female,22.0


In [5]:
#年齢の欠損値を男女の平均年齢で補間
age_train_mean = train_df.groupby('Sex').Age.mean()
 
def fage(x):
    if x.Sex == 'male':
        return round(age_train_mean['male'])
    if x.Sex == 'female':
        return round(age_train_mean['female'])
 
train_df.Age.fillna(train_df[train_df.Age.isnull()].apply(fage,axis=1),inplace=True)

age_test_mean = test_df.groupby('Sex').Age.mean()
 
def fage(x):
    if x.Sex == 'male':
        return round(age_test_mean['male'])
    if x.Sex == 'female':
        return round(age_test_mean['female'])
 
test_df.Age.fillna(test_df[test_df.Age.isnull()].apply(fage,axis=1),inplace=True)

In [6]:
#クロス集計
sex_ct = pd.crosstab(train_df['Sex'], train_df['Survived'])
sex_ct

Survived,0,1
Sex,,
female,81,233
male,468,109


In [7]:
#Femaleカラムを追加し、Sex要素のmale/femaleを1/0に変換して、要素として追加する
train_df['Female'] = train_df['Sex'].map( {'male': 0, 'female': 1} ).astype(int)
test_df['Female'] = test_df['Sex'].map( {'male': 0, 'female': 1} ).astype(int)
 
train_df.head()

,PassengerId,Survived,Pclass,Sex,Age,Female
0,1,0,3,male,22.0,0
1,2,1,1,female,38.0,1
2,3,1,3,female,26.0,1
3,4,1,1,female,35.0,1
4,5,0,3,male,35.0,0


In [8]:
#クロス集計
pclass_ct = pd.crosstab(train_df['Pclass'], train_df['Survived'])
pclass_ct

Survived,0,1
Pclass,,
1,80,136
2,97,87
3,372,119


In [9]:
#Pclassをダミー変数で分ける
pclass_train_df  = pd.get_dummies(train_df['Pclass'],prefix='Class')
pclass_test_df  = pd.get_dummies(test_df['Pclass'],prefix='Class')
 
pclass_train_df.head()

,Class_1,Class_2,Class_3
0,0,0,1
1,1,0,0
2,0,0,1
3,1,0,0
4,0,0,1


In [10]:
#Class_3を削除
pclass_train_df = pclass_train_df.drop(['Class_3'], axis=1)
pclass_test_df = pclass_test_df.drop(['Class_3'], axis=1)
 
#Class_1,Class_2カラムを追加
train_df = train_df.join(pclass_train_df)
test_df = test_df.join(pclass_test_df)

In [11]:
#訓練用
train_df.head()

,PassengerId,Survived,Pclass,Sex,Age,Female,Class_1,Class_2
0,1,0,3,male,22.0,0,0,0
1,2,1,1,female,38.0,1,1,0
2,3,1,3,female,26.0,1,0,0
3,4,1,1,female,35.0,1,1,0
4,5,0,3,male,35.0,0,0,0


In [12]:
#テスト用
test_df.head()

,PassengerId,Pclass,Sex,Age,Female,Class_1,Class_2
0,892,3,male,34.5,0,0,0
1,893,3,female,47.0,1,0,0
2,894,2,male,62.0,0,0,1
3,895,3,male,27.0,0,0,0
4,896,3,female,22.0,1,0,0


In [13]:
X = train_df.drop(['PassengerId','Survived','Pclass','Sex'],axis=1)
y = train_df.Survived
 
#モデルの生成
clf = LogisticRegression(solver="lbfgs")
 
#学習
clf.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [14]:
clf.predict_proba(X)

array([[0.8825431 , 0.1174569 ],
       [0.10700253, 0.89299747],
       [0.41681246, 0.58318754],
       ...,
       [0.4324001 , 0.5675999 ],
       [0.49391267, 0.50608733],
       [0.91179656, 0.08820344]])

In [15]:
#学習したモデルの精度
clf.score(X,y)

0.7991021324354658

In [16]:
#モデルに伴う生存率の予測値
predict_y = clf.predict(X)
 
#実際の値と予測値の比率
accuracy_score(y, predict_y)

0.7991021324354658

In [21]:
#変数名とその係数を格納するデータフレーム
coeff_df = DataFrame([X.columns, clf.coef_[0]]).T
coeff_df

,0,1
0,Age,-0.0319035
1,Female,2.48022
2,Class_1,2.1687
3,Class_2,1.10217


In [22]:
#femaleのオッズ比
print(np.exp(2.46126))

11.719568897704196


In [23]:
test_df.head()

,PassengerId,Pclass,Sex,Age,Female,Class_1,Class_2
0,892,3,male,34.5,0,0,0
1,893,3,female,47.0,1,0,0
2,894,2,male,62.0,0,0,1
3,895,3,male,27.0,0,0,0
4,896,3,female,22.0,1,0,0


In [24]:
#テストデータから生存者を予測
test_predict = clf.predict(test_df.drop(['PassengerId','Pclass','Sex'],axis=1))
print(test_predict)

[0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 0 0 0 1 1 1 0 1 0 0 0 0 0 1 1 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0
 1 0 0 1 0 1 0 0 0 0 0 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 0 1 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 1 0 1 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 0 0
 0 1 1 1 1 1 0 1 0 0 0]


In [25]:
test_probability = clf.predict_proba(test_df.drop(['PassengerId','Pclass','Sex'],axis=1))
print(test_probability)

[[0.91800388 0.08199612]
 [0.58275681 0.41724319]
 [0.89941135 0.10058865]
 [0.89809671 0.10190329]
 [0.38615909 0.61384091]
 [0.85339846 0.14660154]
 [0.44812274 0.55187726]
 [0.73926958 0.26073042]
 [0.35638243 0.64361757]
 [0.87919538 0.12080462]
 [0.90652834 0.09347166]
 [0.64878872 0.35121128]
 [0.06912018 0.93087982]
 [0.90226112 0.09773888]
 [0.13769121 0.86230879]
 [0.18214919 0.81785081]
 [0.79072598 0.20927402]
 [0.87919538 0.12080462]
 [0.42458753 0.57541247]
 [0.56716502 0.43283498]
 [0.71112424 0.28887576]
 [0.83229645 0.16770355]
 [0.08494664 0.91505336]
 [0.45416208 0.54583792]
 [0.14152315 0.85847685]
 [0.94834021 0.05165979]
 [0.06709543 0.93290457]
 [0.8841866  0.1158134 ]
 [0.61163776 0.38836224]
 [0.90652834 0.09347166]
 [0.85910294 0.14089706]
 [0.7267844  0.2732156 ]
 [0.47189265 0.52810735]
 [0.44812274 0.55187726]
 [0.52579293 0.47420707]
 [0.87046472 0.12953528]
 [0.44812274 0.55187726]
 [0.37862468 0.62137532]
 [0.89210705 0.10789295]
 [0.90652834 0.09347166]


In [26]:
test_probability[:,0]

array([0.91800388, 0.58275681, 0.89941135, 0.89809671, 0.38615909,
       0.85339846, 0.44812274, 0.73926958, 0.35638243, 0.87919538,
       0.90652834, 0.64878872, 0.06912018, 0.90226112, 0.13769121,
       0.18214919, 0.79072598, 0.87919538, 0.42458753, 0.56716502,
       0.71112424, 0.83229645, 0.08494664, 0.45416208, 0.14152315,
       0.94834021, 0.06709543, 0.8841866 , 0.61163776, 0.90652834,
       0.85910294, 0.7267844 , 0.47189265, 0.44812274, 0.52579293,
       0.87046472, 0.44812274, 0.37862468, 0.89210705, 0.90652834,
       0.92818218, 0.52579293, 0.93232107, 0.21241441, 0.13028889,
       0.89210705, 0.64148508, 0.90652834, 0.19468438, 0.4957907 ,
       0.47797489, 0.74537188, 0.16390328, 0.08011644, 0.76310698,
       0.83670243, 0.91919663, 0.89210705, 0.90652834, 0.1010567 ,
       0.86497244, 0.77444743, 0.86865541, 0.38615909, 0.39195644,
       0.21241441, 0.35638243, 0.65602335, 0.53374036, 0.19468438,
       0.40138937, 0.87919538, 0.44024643, 0.5138485 , 0.09819

In [27]:
test_pro_pre = pd.DataFrame({"Probability":test_probability[:,1], 'prediction':test_predict})
print(test_pro_pre)

     Probability  prediction
0       0.081996           0
1       0.417243           0
2       0.100589           0
3       0.101903           0
4       0.613841           1
5       0.146602           0
6       0.551877           1
7       0.260730           0
8       0.643618           1
9       0.120805           0
10      0.093472           0
11      0.351211           0
12      0.930880           1
13      0.097739           0
14      0.862309           1
15      0.817851           1
16      0.209274           0
17      0.120805           0
18      0.575412           1
19      0.432835           0
20      0.288876           0
21      0.167704           0
22      0.915053           1
23      0.545838           1
24      0.858477           1
25      0.051660           0
26      0.932905           1
27      0.115813           0
28      0.388362           0
29      0.093472           0
..           ...         ...
388     0.120805           0
389     0.181491           0
390     0.5299

In [28]:
test_df = pd.read_csv('test.csv')
result_df = pd.DataFrame({'PassengerId': test_df['PassengerId'], 'Survived':np.array(test_predict)})
result_df.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
